In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [3]:
value = """
def metric_name(output_name, set_name, metric_name):
    parts = [set_name, output_name, metric_name]
    name = reduce(lambda x, y: x + '_' + y if x else y , parts, '')
    return name

def plot_history(h, outputs=['']):
    fig, axes = plt.subplots(len(outputs), 2, figsize=(12, 5*len(outputs)), facecolor='w', edgecolor='k')

    def plot(ax, output, metric):
        label = '{} {}'.format(output, metric).strip().capitalize()
        data = h[metric_name(output, '', metric)]
        ax.plot(data, label=label)
        ax.scatter([np.argmax(data), np.argmin(data)], [max(data), min(data)])

        vlabel = '{} validation {}'.format(output, metric).strip().capitalize()
        data = h[metric_name(output, 'val', metric)]
        ax.plot(data, label=vlabel)
        ax.scatter([np.argmax(data), np.argmin(data)], [max(data), min(data)])

        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric.capitalize())
        ax.set_title('{} {}'.format(output, metric).strip().capitalize())
        ax.legend()

    for idx,o in enumerate(outputs):
        if len(outputs) == 1:
            plot(axes[0], o, 'loss')
            plot(axes[1], o, 'accuracy')
        else:
            plot(axes[idx, 0], o, 'loss')
            plot(axes[idx, 1], o, 'accuracy')

    plt.tight_layout()
    # fig.legend()

def run_summary(h, outputs=['']):
    plot_history(h, outputs)
    for o in outputs:
        print(o + ': ' if o else '', max(h[metric_name(o, '', 'accuracy')]), '|', max(h[metric_name(o, 'val', 'accuracy')]))
"""

key = 'run_summary'

print(shelf.get(key))
shelf[key] = value
shelf.sync()


def plot_history(h):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5), facecolor='w', edgecolor='k')

    axes[0].plot(h['loss'], label='Loss')
    axes[0].plot(h['val_loss'], label='Validation Loss')

    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Losses')

    axes[1].plot(h['accuracy'], label='Accuracy')
    axes[1].plot(h['val_accuracy'], label='Validation Accuracy')

    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Accuracies')

    fig.legend()

def run_summary(h):
    plot_history(h)
    print(max(h['accuracy']), '|', max(h['val_accuracy']))



## Enumerate

In [4]:
keys = list(shelf.keys())
print(len(keys), keys)

6 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment']


## Query

In [6]:
print(shelf['run_summary'])


def metric_name(output_name, set_name, metric_name):
    parts = [set_name, output_name, metric_name]
    name = reduce(lambda x, y: x + '_' + y if x else y , parts, '')
    return name

def plot_history(h, outputs=['']):
    fig, axes = plt.subplots(len(outputs), 2, figsize=(12, 5*len(outputs)), facecolor='w', edgecolor='k')

    def plot(ax, output, metric):
        label = '{} {}'.format(output, metric).strip().capitalize()
        data = h[metric_name(output, '', metric)]
        ax.plot(data, label=label)
        ax.scatter([np.argmax(data), np.argmin(data)], [max(data), min(data)])

        vlabel = '{} validation {}'.format(output, metric).strip().capitalize()
        data = h[metric_name(output, 'val', metric)]
        ax.plot(data, label=vlabel)
        ax.scatter([np.argmax(data), np.argmin(data)], [max(data), min(data)])

        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric.capitalize())
        ax.set_title('{} {}'.format(output, metric).strip().capitalize())
      